# GPT-2 QLoRA + SEAL (Colab)

This notebook sets up the environment on Google Colab, installs dependencies, clones the repository, and walks through:
- Training GPT-2 Small with QLoRA (LoRA on 4-bit)
- Evaluating perplexity
- Running SEAL-style adaptation

Use Runtime → Change runtime type → GPU.

In [ ]:
# Check GPU availability and basic environment
import torch
print('Torch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
!nvidia-smi || true

In [ ]:
%%capture
# Install libraries (keep Colab's PyTorch)
!pip -q install -U "transformers>=4.40" "datasets>=2.15" "peft>=0.8.0" "accelerate>=0.27.0" "bitsandbytes>=0.44.0"

In [ ]:
# Clone this repository and change directory
import os
if not os.path.exists('/content/Ai-Research'):
    !git clone https://github.com/Hussain0327/Ai-Research.git /content/Ai-Research
%cd /content/Ai-Research
!ls -la

## Optional: Mount Google Drive
Uncomment below to save checkpoints/results to Drive.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# Example: use a Drive path
# OUTPUT_DIR = '/content/drive/MyDrive/ai_research/checkpoints/gpt2_qlora_demo'
# !mkdir -p $OUTPUT_DIR

## Train QLoRA on GPT-2 (TXT sample)
Small smoke test on the bundled text sample.

In [ ]:
!python -m src.gpt2_qlora.train \
  --model_name gpt2 \
  --train_file data/sample/train.txt \
  --output_dir checkpoints/gpt2_qlora_demo \
  --lora_r 8 --lora_alpha 16 --lora_dropout 0.05 \
  --block_size 128 --batch_size 2 --epochs 1 --lr 1e-4

## Evaluate perplexity (TXT sample)

In [ ]:
!python -m src.gpt2_qlora.eval \
  --model_name gpt2 \
  --adapter_dir checkpoints/gpt2_qlora_demo \
  --eval_file data/sample/train.txt \
  --block_size 128 --max_batches 50

## Train QLoRA on JSONL (repo `text.JSONL`)
Uses the `text` field from the provided JSONL file.

In [ ]:
!python -m src.gpt2_qlora.train \
  --model_name gpt2 \
  --train_file text.JSONL \
  --data_format jsonl --text_key text \
  --output_dir checkpoints/gpt2_qlora_jsonl \
  --lora_r 8 --block_size 128 --epochs 1

## SEAL-style adaptation (LoRA on GPT-2)
Runs short inner-loop updates and logs metrics to JSONL.

In [ ]:
!python -m src.seal.hf_seal \
  --model_name gpt2 \
  --baseline_adapter checkpoints/gpt2_qlora_demo \
  --save_dir checkpoints/gpt2_seal/adapt \
  --results results/gpt2_seal/adapt.jsonl \
  --inner_steps 1 3 10 \
  --lora_rank 4 8 \
  --train_file data/sample/train.txt \
  --val_file data/sample/train.txt \
  --block_size 128

## View adaptation logs

In [ ]:
!tail -n 50 results/gpt2_seal/adapt.jsonl || echo 'No logs yet'